# Music Charts

In this example, we will extract information about music charts from tables on Wikipedia.
We will populate the Wikidata predicate "charted in" ([P2291](https://www.wikidata.org/wiki/Property:P2291)), which often occurs with the qualifiers "point in time" ([P585](https://www.wikidata.org/wiki/Property:P585)) and "ranking" ([P1352](https://www.wikidata.org/wiki/Property:P1352)).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import takco

pred_charted_in = "http://www.wikidata.org/prop/direct/P2291"
pages = takco.wiki(pred=pred_charted_in, sample=10)
print(f"Downloaded {len(list(pages))} pages")

extracted = takco.TableSet.extract(source = pages)

print(f"Processed {len(list(extracted))} tables")
takco.preview(extracted)

Downloaded 10 pages
Processed 60 tables


?,0,1
,Source,Rating
,Aggregate scores,Aggregate scores
,Metacritic,75/100 (24 reviews)
?,0,1
,Source,Rating
,Review scores,Review scores
,The A.V. Club,B+
,AllMusic,3.5/5 stars
,Entertainment Weekly,A−
,Paste,75/100


In [3]:
reshaped = takco.TableSet.reshape(extracted)
print(f"Processed {len(list(reshaped))} tables")
takco.preview(reshaped, nrows=5, ntables=25)

Processed 51 tables


?,0,1
,Source,Rating
,Aggregate scores,Aggregate scores
,Metacritic,75/100 (24 reviews)
?,0,1
,Source,Rating
,Review scores,Review scores
,The A.V. Club,B+
,AllMusic,3.5/5 stars
,Entertainment Weekly,A−
,Paste,75/100


In [4]:
clustered = takco.TableSet.cluster(reshaped, addcontext = ["pgTitle"], matchers=[])

print(f"Processed {len(list(clustered))} tables")
takco.preview(clustered, nrows=5, ntables=25)

Processed 20 tables


In [5]:
linked = takco.TableSet.link(clustered, lookup_cells=False, linker_config = {}, lookup_config = {
    'class': 'SQLiteWikiLookup',
    'sqlitedb': '../../data/wdid_wpname.sqlitedb',
    'baseuri': 'http://www.wikidata.org/entity/',
    'fallback': {'class': 'MediaWikiAPI'},
})

print(f"Processed {len(list(linked))} tables")
takco.preview(linked, nrows=5, ntables=25)

Processed 20 tables


In [6]:
searcher_config = {
    'class': "RDFSearcher",
    'encoding': "wikidata",
    'typeProperties': ["http://www.wikidata.org/prop/direct/P31"],
    'statementURIprefix': "http://www.wikidata.org/entity/statement/",
    'store': {
        'class': "HDTStore",
        'path': "/export/scratch1/home/kruit/scratch/downloads/wikidata/wikidata20200309.hdt",
    }
}
integrated = takco.TableSet.integrate(linked, pfd_threshold = 0.95, searcher_config=searcher_config)
print(f"Processed {len(list(integrated))} tables")

integrated = sorted(integrated, key=lambda table: -table.get('numDataRows', 0))
takco.preview(integrated, nrows=5, ntables=25)

Processed 20 tables


?,0,1,2,3
∈,Thing,decimal,dateTime,Thing
0,,P1352,P585,P2291
,_Page,Peak position,Variable,Chart
,The Best Damn Thing,2,2007,Australian Albums ( ARIA )
,The Best Damn Thing,1,2007,Austrian Albums ( Ö3 Austria )
,The Best Damn Thing,9,2007,Belgian Albums ( Ultratop Flanders)
,The Best Damn Thing,4,2007,Belgian Albums ( Ultratop Wallonia)
,The Best Damn Thing,1,2007,Canadian Albums ( Billboard )
?,0,1,2,3
∈,Thing,decimal,dateTime,string
